### **This is LEADS_redcapfill (script 2A and 2B); to include all the latest subjects, this script must be run after running leads_unpack (script 1A and 1B) and requires the completion of manual edits (THE RECON-ALL JOBS MUST BE COMPLETED). This script 2A will re-run the GLM and re-generate the large table of roi summary stats.**

In [2]:
# Loading data into csv for redcap input
import re
import getpass
import pandas as pd
import os
import pydicom
from pydicom.tag import Tag
import datetime
import glob
import math
import csv
from redcap import Project, RedcapError
import subprocess
from paramiko import SSHClient
from os import system
from nipype.interfaces.freesurfer import MRISPreproc
from nipype.interfaces.freesurfer import SurfaceSmooth
from nipype.interfaces.freesurfer import GLMFit

today = str(datetime.date.today())

# make a dictionary for what number ID corresponds to what site:
sites = {
    "360": "MGHMartinos4", 
    "011": "WashUSL", # CCIR PRISMA (WashUSL)
    "022": "UPenn", # 'HUP' (UPenn)
    "037": "IndianaU", # INDIANA UNIVERSITY
    "073": "UCSF", # Neuroscience Imaging Center
    "941": "BrownU", # BROWN UNIVERSITY MRF / # ButlerHosp / BHMAP
    "007": "Mayo", #MayoCRochester or MayoCJacksonville?
    "010": "CPMC", # CA Pacific medical center?
    "018": "JHopkins",
    "032": "Emory",
    "035": "UCLA",
    "036": "MCJ",
    "067": "NorthWesternU",
    "177": "HoustonMNI",
    # Columbia,?
    # UWisconsin 127 (no longer involved with LEADS)
}

In [3]:
dicom = '/autofs/cluster/animal/users/rezzo/PPA/PPA_baseline/recons/SFRE_M_47_20160610/dicoms/3/DICOM/1.2.124.113532.80.22200.7094.20160607.154802.184937459-3-9-sesbkh.dcm'
ds = pydicom.read_file(dicom)
ds
#TR = str(ds[0x18, 0x80].value)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM']
(0008, 0012) Instance Creation Date              DA: '20160610'
(0008, 0013) Instance Creation Time              TM: '101235.218000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.32.35022.2016061010123094557312784
(0008, 0020) Study Date                          DA: '20160610'
(0008, 0021) Series Date                         DA: '20160610'
(0008, 0022) Acquisition Date                    DA: '20160610'
(0008, 0023) Content Date                        DA: '20160610'
(0008, 0030) Study Time                          TM: '100301.609000'
(0008, 0031) Series Time                         TM: '101226.921000'
(0008, 0032) Acquisition Time                    TM: '100621.075000'
(0008, 0033) Content Time                        TM: '101235.21800

In [2]:
def passprompt():
    USER= input("Please enter your USERNAME for launchpad acess: ")
    print('PASSWORD: ')
    PASS= getpass.getpass()
    return USER, PASS

In [3]:
[USER, PASS] =passprompt()

Please enter your USERNAME for launchpad acess: rje11
PASSWORD: 
········


In [838]:
# run ANALYZE.SUBJECTS.sh, which called SUBJECTSFILE
# MAKE SURE SUBJECTSFILE IS UP TO DATE WITH ALL MANUALLY EDITED, REPROCESSED RECONS!!!!!!
host="launchpad"
user=USER
pw=PASS
client=SSHClient()
client.load_system_host_keys()
client.connect(host,username=user,password=pw, look_for_keys=False)
stdin, stdout, stderr = client.exec_command('(cd /autofs/cluster/animal/scan_data/leads/analyses; ./ANALYZESUBJECTS.sh)')
# print("stderr: ", stderr.readlines())
# print("pwd: ", stdout.readlines())

In [57]:
# PREPROCESS (but no smoothing)

# THESE MUST BE UPDATED SOMEHOW... connect to demog if using that?
amy_pos_cont = '/autofs/cluster/animal/scan_data/leads/analyses/ABpos_control.fsgd'
amy_neg_cont = '/autofs/cluster/animal/scan_data/leads/analyses/ABneg_control.fsgd'
os.environ['SUBJECTS_DIR'] = '/autofs/cluster/animal/scan_data/leads/analyses/' # use this instead
contrastfile = '/autofs/cluster/animal/scan_data/leads/analyses/group.diff.mtx'

# create index for both hemispheres, and both conditions (AMY+ vs control, and AMY- vs control) 
index =  [[amy_pos_cont, 'lh'],[amy_pos_cont, 'rh'],[amy_neg_cont, 'lh'],[amy_neg_cont, 'rh']]

os.environ['SUBJECTS_DIR'] = '/autofs/cluster/animal/scan_data/leads/analyses/' # use this instead
os.chdir('/autofs/cluster/animal/scan_data/leads/analyses/')

for metric in index:

    preproc = MRISPreproc()
    preproc.inputs.fsgd_file = metric[0]
    preproc.inputs.target = 'fsaverage'
    preproc.inputs.hemi = metric[1]
    preproc.inputs.surf_measure = 'thickness'
    preproc.inputs.out_file = '/autofs/cluster/animal/scan_data/leads/analyses/'+metric[1]+'.'+metric[0][-18:-5]+'.thickness.00.mgz'
    #preproc.inputs.environ = {"SUBJECTS_DIR": "/autofs/cluster/animal/scan_data/leads/analyses/"}
    #preproc.inputs.subjects_dir = "/autofs/cluster/animal/scan_data/leads/analyses/" # does not work? not sure why
    system(preproc.cmdline)
    

/autofs/cluster/animal/scan_data/leads/analyses/ABpos_control.fsgd and lh
/autofs/cluster/animal/scan_data/leads/analyses/ABpos_control.fsgd and rh
/autofs/cluster/animal/scan_data/leads/analyses/ABneg_control.fsgd and lh
/autofs/cluster/animal/scan_data/leads/analyses/ABneg_control.fsgd and rh


In [68]:
# SMOOTH (wraps mri_surf2surf)

for metric in index:
    smoother = SurfaceSmooth()
    smoother.inputs.hemi = metric[1]
    smoother.inputs.subject_id = 'fsaverage'
    smoother.inputs.in_file = metric[1]+'.'+metric[0][-18:-5]+'.thickness.00.mgz'
    smoother.inputs.fwhm = 10
    smoother.inputs.cortex = True # defaults to 1
    smoother.inputs.out_file = metric[1]+'.'+metric[0][-18:-5]+'.thickness.10.mgz'
    smoother.inputs.subjects_dir = "/autofs/cluster/animal/scan_data/leads/analyses/"
    system(smoother.cmdline)

/autofs/cluster/animal/scan_data/leads/analyses/ABpos_control.fsgd and lh
/autofs/cluster/animal/scan_data/leads/analyses/ABpos_control.fsgd and rh
/autofs/cluster/animal/scan_data/leads/analyses/ABneg_control.fsgd and lh
/autofs/cluster/animal/scan_data/leads/analyses/ABneg_control.fsgd and rh


In [79]:
# # GLM analysis (mri_glmfit)

for metric in index:
    glmfit = GLMFit()
    glmfit.inputs.in_file = metric[1]+'.'+metric[0][-18:-5]+'.thickness.10.mgz'
    glmfit.inputs.fsgd = (metric[0], 'dods')
    glmfit.inputs.contrast = contrastfile
    glmfit.inputs.cortex = True # defaults to 1
    glmfit.inputs.subjects_dir = "/autofs/cluster/animal/scan_data/leads/analyses/"
    glmfit.inputs.subject_id = 'fsaverage' # not sure if this is correct?
    glmfit.inputs.hemi = metric[1] # not sure if this is correct?
    glmfit.inputs.surf = True
    glmfit.inputs.glm_dir = "/autofs/cluster/animal/scan_data/leads/analyses/"+metric[1]+'.'+metric[0][-18:-5]+'.glmdir'
    system(glmfit.cmdline)


In [85]:
# screen shots??

shots = fs.SurfaceSnapshots(subject_id="fsaverage", hemi="lh", surface="inflated")
shots.inputs.overlay = '/autofs/cluster/animal/scan_data/leads/analyses/lh.ABneg_control.glmdir/group.diff/sig.mgh'
shots.inputs.overlay_range = (1.64, 3)
shots.inputs.invert_overlay = True
shots.inputs.subjects_dir = '/autofs/cluster/animal/scan_data/leads/analyses/'
shots.inputs.screenshot_stem = '/autofs/cluster/animal/scan_data/leads/analyses/screenshots'
system(shots.cmdline)

# not sure how to make use of this yet..

256

### **This is LEADS_redcapfill (script 2B). This script will pull the stats output, dicom info, and info from spreadsheets that are necessary to create the data matrix which will be transformed and imported into RedCap.**

In [827]:
analysesdir = "/autofs/cluster/animal/scan_data/leads/analyses/"
recondir = "/autofs/cluster/animal/scan_data/leads/recon/"

# load in the instrument fields
import_instrument = pd.read_csv('/autofs/cluster/animal/scan_data/leads/redcap_instruments/LEADS_ImportTemplate_2019-02-07.csv')
fieldlist = list(import_instrument.columns.values) # all fields

groupinfo = '/autofs/cluster/animal/scan_data/leads/analyses/demographics.csv'
grouplabels = pd.read_csv(groupinfo)

statswhole = analysesdir+'/leads_aseg_stats.txt'
statslsurf = analysesdir+'/lh_leads_aparc_stats.txt'
statsrsurf = analysesdir+'/rh_leads_aparc_stats.txt'

whole = pd.read_csv(statswhole,sep='\t')
left = pd.read_csv(statslsurf,sep='\t')
right = pd.read_csv(statsrsurf,sep='\t')

tmpstats = pd.merge(whole, left, right_index=True, left_index=True)
totalstats = pd.merge(tmpstats, right, right_index=True, left_index=True)

demListOfSeries = []
visitListOfSeries = []

# conversion table of aparc / aseg names
roikeys = '/autofs/cluster/animal/scan_data/leads/redcap_instruments/LEADS_roiname_conversion.csv'
with open(roikeys, mode='r') as infile:
    reader = csv.reader(infile)
    roidict = {rows[0]:rows[1] for rows in reader}

In [828]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
def cutid(sessid):
    find = re.compile(r"^[^_]*")
    subjid = re.search(find, sessid).group(0)
    return subjid

def group(subject):
    groupspec = (grouplabels.loc[grouplabels['Subject'] == subject, 'Group'].values[0])
    amyspec = (grouplabels.loc[grouplabels['Subject'] == subject, 'AMYLEVEL'].values[0])
    return groupspec, amyspec

def convsex(S):
    if S == "O":
        S = ""
    elif S == "M":
        S = "1"
    elif S == "F":
        S = "0"
    return S

def convgroup(G):
    try:
        value = math.isnan(float(G))
        if value:
            G = ""
    except(ValueError):
        pass
    if G == "PT":
        G = "0"
    elif G == "CN":
        G = "1"
    return G

def converting2dic(path):
    with open(pathtomaster) as fp:
        datalist = []
        for cnt, line in enumerate(fp):
            newline = line.split(",")
            if cnt == 0:
                header = newline
            else:
                dictionary = dict(zip(header,newline))
                datalist.append(dictionary)
    return datalist

In [835]:
folders = [x for x in os.listdir(analysesdir) if x.startswith("LDS")]

#folders = ['LDS0370012_20180824','LDS0370005_20180802']

dem_df = pd.DataFrame(columns=fieldlist, index=range(len(folders)))
visit_df = pd.DataFrame(columns=fieldlist, index=range(len(folders)))
a=0

#for sessid in folders:
for element in range(0,len(folders)):
    sessid = folders[element]
    print(sessid)

    fs_sessid = totalstats.loc[totalstats['Measure:volume'] == sessid] # loading roi stats for particular subject
    
    dicomlist = '/autofs/cluster/animal/scan_data/leads/recon/'+sessid+'/dicomlist.txt'
    with open(dicomlist) as f:
        firstline = f.readline()[:-1]

    dateshortcut= sessid[-8:-4]+'-'+sessid[-4:-2]+'-'+sessid[-2:]
    dicom = find(firstline,'/autofs/cluster/animal/scan_data/leads/LEADS/'+sessid)
    editline = dicom.split("/")
    sessionserial = editline[-2]
    DICOMPATH = dicom.replace(editline[- 1],"")
    SITE = sites.get(sessid[3:6])

    # dicom info
    ds = pydicom.read_file(dicom)
    TR = str(ds[0x18, 0x80].value)
    SEX = convsex(str(ds[0x10,0x40].value))
    age = ds[0x00101010].value
    AGE = str(int(age[:-1]))
    # COIL = ds[0x18, 1250].value  # not found by pydicom
    SCANDATE = ds[0x00080022].value
    SUBJECTID = cutid(folders[element])
    [G, A] = group(SUBJECTID)
    GROUP = convgroup(G)
    AMYSTATUS = A
    
    # get FS version
    element = '/autofs/cluster/animal/scan_data/leads/recon/'+sessid
    prefixed = [filename for filename in os.listdir(element) if filename.startswith("edit")]
    editstring=(prefixed[0].replace('edit.','').replace(',',' ').split())[0]
    findversion = re.compile(r"^[^_]*")
    tmpV = re.search(findversion, editstring).group(0)
    FSVERSION = re.sub(r'[A-Z]+', '', tmpV, re.I)
    
    dem_df.loc[a, 'record_id'] = SUBJECTID
    #dem_df.loc[a, "redcap_repeat_instrument"] = ""
    #dem_df.loc[a,'redcap_repeat_instance'] = ''
    dem_df.loc[a,'site_name'] = SITE
    dem_df.loc[a,'sex'] = SEX
    dem_df.loc[a,'group'] = GROUP
    dem_df.loc[a,'notes'] = ""
    dem_df.loc[a,'demographics_complete'] = 1
    
    visit_df.loc[a, 'record_id'] = SUBJECTID
    visit_df.loc[a, "redcap_repeat_instrument"] = "visits"
    visit_df.loc[a,'redcap_repeat_instance'] = "1"
    visit_df.loc[a,'scan_date'] = dateshortcut #convert date to correct format!
    visit_df.loc[a,'age'] = AGE
    visit_df.loc[a,'session_id'] = sessid
    visit_df.loc[a,'dicom_path'] = DICOMPATH
    visit_df.loc[a,'fs_version'] = FSVERSION
    
    
    for key in roidict:
        FSname = roidict.get(key)
        if FSname != 'freesurfer_var':
            try:
                val = fs_sessid[FSname].values[0]
                visit_df.loc[a, key] = val
            except(IndexError):
                pass
            
    print("Updated redcap matrix data for "+sessid+" ...........")        
    a=a+1

dem_df.to_csv('/autofs/cluster/animal/scan_data/leads/redcap_instruments/demographics_'+today+'.csv')
visit_df.to_csv('/autofs/cluster/animal/scan_data/leads/redcap_instruments/visits_'+today+'.csv')
data_for_import = dem_df.append(visit_df)

#MUST SAVE WITHOUT COLUMN!!! OTHERWISE MUST MANUALLY EDIT!
#data_for_import.to_csv('/autofs/cluster/animal/scan_data/leads/redcap_instruments/data_for_import_'+today+'.csv')
data_for_import.to_csv('/autofs/cluster/animal/scan_data/leads/redcap_instruments/data_for_import_'+today+'.csv',mode = 'w', index=False)

LDS0370020_20181212
Updated redcap matrix data for LDS0370020_20181212 ...........
LDS0370001_20180509
Updated redcap matrix data for LDS0370001_20180509 ...........
LDS0370042_20190108
Updated redcap matrix data for LDS0370042_20190108 ...........
LDS0110022_20181129
Updated redcap matrix data for LDS0110022_20181129 ...........
LDS9410025_20181128
Updated redcap matrix data for LDS9410025_20181128 ...........
LDS0370015_20181113
Updated redcap matrix data for LDS0370015_20181113 ...........
LDS0370005_20180802
Updated redcap matrix data for LDS0370005_20180802 ...........
LDS3600043_20190122
Updated redcap matrix data for LDS3600043_20190122 ...........
LDS0110040_20190124
Updated redcap matrix data for LDS0110040_20190124 ...........
LDS0220026_20181109
Updated redcap matrix data for LDS0220026_20181109 ...........
LDS3600030_20181219
Updated redcap matrix data for LDS3600030_20181219 ...........
LDS0110021_20181016
Updated redcap matrix data for LDS0110021_20181016 ...........
LDS0

In [836]:
# connect to redcap (note: exporting means retrieving from redcap; importing means uploading to redcap)

api_url = 'https://redcap.partners.org/redcap/api/'
api_key = '9D668EE32160684D980CD6A95B23CC5B'
# to set specific certificate pathway: verify_ssl='path/to/CA_bundle/' or set to False to ignore SSH verification altogether
project = Project(api_url, api_key) # pycap verifies SSL certificate under the hood (using requests)
# will return a list of dicts from all record IDs with all raw field names as keys
data = project.export_records(format='df')


In [837]:
# Send to REDCAP
pathtomaster = '/autofs/cluster/animal/scan_data/leads/redcap_instruments/data_for_import_'+today+'.csv'
final_dic = converting2dic(pathtomaster)
response = project.import_records(final_dic, date_format='YMD')